In [32]:
using Revise
using Plots, SparseArrays,Interpolations, Parameters, ColorSchemes
includet("Module_Dictyostelium_migration.jl")

# Dynamic simulation of Dictyostelium group chemotaxis.

We describe the Dictyostelium group as a thin, polar active droplet of height $h(y,t)$ characterised by and emergent surface tension, $\kappa$, viscosity, $\eta$ and contractile activity $\xi$. The group migration is directed towards the food source (bacteria--$B(y,t)$) as a result of cells sensing gradients in the concentration of diffusable molecules (chemoattractants $C(y,t)$) that are produced by the bacteria:

\begin{equation}
\frac{\partial C}{\partial t} = D_C \frac{\partial^2 C}{\partial y^2} + \lambda\, (B-C), \quad y\in[0,L_{lawn}].
\end{equation}

Inside the droplets, the flow of cells, $u_c$ is dictated by the interplay of capillary forces -- due to the surface tension $\kappa$ -- and active forces -- due to the pulling of Dictyostelium cells onto each other as they move. This generates a net mass flux that together with proliferation determines the evolution of the group height:

\begin{equation}
\partial_t h+\partial_y\left(M(h)\partial_y \pi\right)= r h,\quad  y\in \left[y_R(t),y_F(t)\right]\subset [0,L_{lawn}]\\ M(h)=h^2\left(\frac{h}{3}+\ell\right), \quad\quad
\pi=\frac{\kappa}{\eta}\frac{\partial^2 h}{\partial y^2}-\frac{\xi}{\eta}\mathcal{S}(\partial_y C),\label{eq:eveolution_H}
\end{equation}

where $\mathcal{S}:\mathbb{R}\rightarrow [0,1]$ captures the strenght of external cues in driving cell movement and the slip length $\ell$ characterises the interaction with the substrate. We assume when $\partial_y C$ is sufficiently strong, dictyostelium cells are perfectly aligned towards the bacteria source; conversely, if the external clue is too low --_i.e._, below their sensitivity range (arbitrary constant, $\alpha$) -- dictyostelium cells move randomly, effectively behaving like a passive droplet. 
Accordingly, we define:

\begin{equation}
\mathcal{S}(\omega) = \frac{\omega^2}{\alpha^2+\omega^2}.
\end{equation}

The location of the front and rear of the group, respectively $y_R$ and $y_F$, correspond to the contact line. Here, we explicitly follow the contact lines, using the approach proposed in . We assume that at equilibrium (passive case), the group wants to be at an equilibrium contact angle, $\theta_{eq}\ll1$. When out of equilibrium, dissipation at the contact line results in reciding/advancing contact angle

\begin{equation}
\dot{y}=\frac{\vec{\nu}}{\eta_{\theta}}\left(\frac{1}{2}\left|\partial_y h\right|^2-\theta_{eq}\right) \label{eq:evolution_Theta}
\end{equation}

where $\vec{\nu}$ is the outer normal to the boundary and $\eta_\theta$ capture the friction coefficient at the contact lines. 

-----------------------------------------------------------------------------------------------------------------------------
### Modelling the bacteria

We apply two strategies to modelling the bacteria:
- *(M1)*: we assume that the bacteria are rapidly depleted by the Dictyo cells and they only accumulate on the RHS of the advancing group
\begin{equation}
B(y,t) = \begin{cases}
1, \quad y>y_F(t),\\
0, \quad y<y_F(t).
\end{cases}
\end{equation}
- *(M2)*: we explicitly model the evolution of the bacteria concentration as it is eaten up by the Dictyostelium cells

\begin{equation}
\partial_t B(y,t) = D_B \frac{\partial^2 B}{\partial y^2} + E h B, \quad y\in[0,L_{lawn}].
\end{equation}

To simulate (M1) it is sufficient to input a negative value for $E$.

----------------------------------------------------------------------------------------------------------------------------
### Numerical implementation

At any given time $t$, the dictyostelium solution $\mathcal{D}(\cdot,t):[0,L_{lawn}]\rightarrow \mathbb{R}_0^+$ consists of the union of $N(t)$ disjoint thin droplets each characterised by its heigth-- $h_n$ -- and the corresponding domain of definition $\mathcal{I}_n=\left[y_R^n(t),y_F^n(t)\right]$. The indexing is such that $y_F^{n}(t)<y_R^{n+1}(t)$ for all $t$ and for $n=1,\ldots,N(t)-1$. Consequently, coupling between the droplets is only inderect via their impact on the scalar fields $C$ and $B$. Splitting is implemented by resection of mass whenever the a droplet height drops below a threshold $\delta=0.025$. 

We decouple the chemoattractant and the Dictyostelium group problem by using a semi-implicit scheme. At each iteration, given the solution at time $t$:

- we solve for $\mathcal{D}(\cdot,t+\delta t)$ fixing $C=C(t)$ -- we update each disjoint droplet independently using the numerical scheme for thin-film free boundary problem with dynamic contact angle proposed in <a href="https://doi.org/10.1063/1.5040985">(Peschka, 2018)</a>

- we solve for $B(\cdot,t+\delta t)$ using the updated solution $\mathcal{D}(t+\delta t)$ -- for model *(M2)* we use a simple central finite-different scheme in space treating $B$ implicitly in time.

- we solve for $C(\cdot,t+\delta t)$ using the updated solution $B(\cdot,t+\delta t)$ -- we use a simple central finite-different scheme in space treating $B$ implicitly in time.
----------------------------------------------------------------------------------------------------------------------------
### Model parameters

We rescale the system as in so that $\theta_{eq}=1$, $B(L_{lawn})=1$, $C(L_{lawn})=1$ and $\frac{\kappa}{\eta}=1$ and $\frac{\xi}{\eta}=4$. This leaves the following unknown parameters

- $\ell$ $\rightarrow$ L_slip 
- $D_C$ $\rightarrow$ Diffusion_coeff_C
- $\lambda$ $\rightarrow$ decay_C
- $D_B$ $\rightarrow$ Diffusion_coeff_B
- $E$ $\rightarrow$ consumption_B (if negative modelling strategy M1 is adopted to simulate the bacteria lawn)
- $r$ $\rightarrow$ proliferation_rate
- $\alpha$ $\rightarrow$ saturation_alignment
- $\eta_\theta$ $\rightarrow$ η_slip

----------------------------------------------------------------------------------------------------------------------------
### Simulation parameters
- initial_time $\rightarrow$ time at which the simulation start ($t_0$)
- final_time $\rightarrow$ time at which the simulation stop ($t_{end}$)
- time_step $\rightarrow$ time step used to advance the solution in time ($dt$)
- initial_group_length $\rightarrow$ initial size of the dictostelium group ($L_0$); default initial condition: $h(y,t_0)=y(L_0-y)$ for $y\in(0,L_0)$
- npoint_droplet $\rightarrow$ number of vertices for the droplet mesh 
- domain_size_lawn $\rightarrow$ size of the lawn ($L_{lawn}/2\gg L_0$): $\Omega=(-L_{lawn}/2,L_{lawn}/2)$
- n_point_lawn $\rightarrow$ number of vertices for the lawn mesh 
- save_every $\rightarrow$ step at which we save the solution $\left\{t_n=\mbox{save_every}*n*dt\right\}_{n=0}$
- plot_every $\rightarrow$ step at which we plot the solution $\left\{t_n=\mbox{plot_every}*n*dt\right\}_{n=0}$


In [ ]:
physical_parameters=(proliferation_rate=0.001,L_slip=0.25,consumption_B=0.1,Diffusion_coeff_B=0.0001,Diffusion_coeff_C=0.01,decay_C=0.1,saturation_alignment=0.1,η_slip=0.005)
simulation_parameters=(initial_time=0,plot_every=1000,initial_group_length=2.0,final_time=3000.,time_step=0.025,npoint_droplet=200,npoint_lawn=1000,domain_size_lawn=40.,save_every=10)
solution=Dynamics_Dictyostelium_group.time_dynamics(physical_parameters,simulation_parameters,"C:/Users/Giulia/Desktop/simulation_results_dictyo")